In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns

In [4]:
data = pd.read_csv('./data/wine_test.csv')
data.head()

,와인_종류,알코올_도수,사과산_함량,재_함량,재의_알칼리도,마그네슘_함량,총_페놀_함량,플라보노이드_함량,비플라보노이드_페놀_함량,프로안토시아닌_함량,색_강도,색조,희석_와인의_투과율_OD280_OD315,프롤린_함량
0,1,14.22,3.99,2.51,13.2,128,3.00,3.04,0.20,2.08,5.10,0.89,3.53,760
1,1,13.48,1.81,2.41,20.5,100,2.70,2.98,0.26,1.86,5.10,1.04,3.47,920
2,2,13.49,1.66,2.24,24.0,87,1.88,1.84,0.27,1.03,3.74,0.98,2.78,472
3,2,12.22,1.29,1.94,19.0,92,2.36,2.04,0.39,2.08,2.70,0.86,3.02,312
4,2,11.76,2.68,2.92,20.0,103,1.75,2.03,0.60,1.05,3.80,1.23,2.50,607


## test 데이터에 train 데이터에 해주었던 작업을 동일하게 적용
* MinMaxScaler로 스케일링
* 홀드아웃 - train, valid, test
* mms = MinMaxScaler()
* mms_X_train = mms.fit_transform(X_train)
* mms_X_valid = mms.transform(X_valid)
* mms_X_test = mms.transform(X_test)

# 모델은 가져갈 수 있는데 scaling은 어떻게 가져올까?
* test 데이터에서 scaler나 vectorizer나 labelencoder를 새로 만들면 안됨!!!
* fit_transform한 것들!!!!

In [5]:
X_test = data.drop('와인_종류',axis=1)
y_test = data['와인_종류']

## 훈련 시 사용한 MinMaxScaler를 joblib로 불러와 스케일링

In [6]:
import joblib

In [7]:
mms = joblib.load('./model/MinMax_info.joblib')
X_test = pd.DataFrame(mms.transform(X_test), columns=X_test.columns, index=X_test.index)

In [8]:
X_test

,알코올_도수,사과산_함량,재_함량,재의_알칼리도,마그네슘_함량,총_페놀_함량,플라보노이드_함량,비플라보노이드_페놀_함량,프로안토시아닌_함량,색_강도,색조,희석_와인의_투과율_OD280_OD315,프롤린_함량
0,0.839474,0.629098,0.618280,0.158537,0.564103,0.673077,0.796875,0.14,0.580420,0.325939,0.333333,0.849421,0.389652
1,0.644737,0.182377,0.564516,0.603659,0.205128,0.557692,0.778125,0.26,0.503497,0.325939,0.455285,0.826255,0.518998
2,0.647368,0.151639,0.473118,0.817073,0.038462,0.242308,0.421875,0.28,0.213287,0.209898,0.406504,0.559846,0.156831
3,0.313158,0.075820,0.311828,0.512195,0.102564,0.426923,0.484375,0.52,0.580420,0.121160,0.308943,0.652510,0.027486
4,0.192105,0.360656,0.838710,0.573171,0.243590,0.192308,0.481250,0.94,0.220280,0.215017,0.609756,0.451737,0.265966
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.563158,0.342213,0.543011,0.573171,0.461538,0.153846,0.059375,0.80,0.363636,0.684300,0.097561,0.111969,0.454325
68,0.797368,0.145492,0.494624,0.329268,0.538462,0.673077,0.837500,0.22,0.587413,0.372867,0.495935,0.918919,0.405821
69,0.365789,0.719262,0.736559,0.969512,0.230769,0.365385,0.512500,0.60,0.451049,0.068259,0.357724,0.691120,0.070331
70,0.163158,0.153689,0.677419,0.939024,0.051282,0.257692,0.350000,0.54,0.321678,0.112628,0.715447,0.725869,0.229588


## train 데이터로 개발한 best 모델을 불러와 예측

In [13]:
y_test = pd.get_dummies(y_test)

In [9]:
from keras.models import load_model

In [14]:
best_model = load_model('./model/wine_best_model.keras')
result = best_model.evaluate(X_test, y_test)
print('loss', result[0], 'accuracy', result[1])

I0000 00:00:1747297381.605126   42241 service.cc:152] XLA service 0x7f4f80003910 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747297381.605163   42241 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2025-05-15 17:23:01.614638: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747297381.643321   42241 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/3 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9688 - loss: 0.0616

I0000 00:00:1747297382.443619   42241 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 242ms/step - accuracy: 0.9705 - loss: 0.0666
loss 0.06556938588619232 accuracy 0.9722222089767456


# 랜덤포레스트 불러와 분석하기

In [15]:
data = pd.read_csv('./data/wine_test.csv')
data.head()

,와인_종류,알코올_도수,사과산_함량,재_함량,재의_알칼리도,마그네슘_함량,총_페놀_함량,플라보노이드_함량,비플라보노이드_페놀_함량,프로안토시아닌_함량,색_강도,색조,희석_와인의_투과율_OD280_OD315,프롤린_함량
0,1,14.22,3.99,2.51,13.2,128,3.00,3.04,0.20,2.08,5.10,0.89,3.53,760
1,1,13.48,1.81,2.41,20.5,100,2.70,2.98,0.26,1.86,5.10,1.04,3.47,920
2,2,13.49,1.66,2.24,24.0,87,1.88,1.84,0.27,1.03,3.74,0.98,2.78,472
3,2,12.22,1.29,1.94,19.0,92,2.36,2.04,0.39,2.08,2.70,0.86,3.02,312
4,2,11.76,2.68,2.92,20.0,103,1.75,2.03,0.60,1.05,3.80,1.23,2.50,607


## test 데이터에 train 데이터에 해주었던 작업을 동일하게 적용
* MinMaxScaler로 스케일링
* 홀드아웃 - train, valid, test
* mms = MinMaxScaler()
* mms_X_train = mms.fit_transform(X_train)
* mms_X_valid = mms.transform(X_valid)
* mms_X_test = mms.transform(X_test)

# 모델은 가져갈 수 있는데 scaling은 어떻게 가져올까?
* test 데이터에서 scaler나 vectorizer나 labelencoder를 새로 만들면 안됨!!!
* fit_transform한 것들!!!!

In [16]:
X_test = data.drop('와인_종류',axis=1)
y_test = data['와인_종류']

In [21]:
data['와인_종류'].value_counts()

와인_종류
2    29
1    24
3    19
Name: count, dtype: int64

## 훈련 시 사용한 MinMaxScaler를 joblib로 불러와 스케일링

In [19]:
import joblib
from sklearn.metrics import classification_report

In [17]:
mms = joblib.load('./model/MinMax_info2.joblib')
X_test = pd.DataFrame(mms.transform(X_test), columns=X_test.columns, index=X_test.index)

In [20]:
rfc = joblib.load('./model/rfc_model.joblib')
pred = rfc.predict(X_test)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           1       1.00      0.92      0.96        26
           2       0.93      1.00      0.96        27
           3       1.00      1.00      1.00        19

    accuracy                           0.97        72
   macro avg       0.98      0.97      0.97        72
weighted avg       0.97      0.97      0.97        72

